In [121]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd #read files
from bs4 import BeautifulSoup
import requests
import re
import csv

In [67]:
df = pd.read_csv('/Users/Tico/Documents/Masters/LaMona/ModelTraining/combinedElements.csv')

In [69]:
df

,Unnamed: 0,html,category,name,price
0,0,"<li class=""mr-3 float-left"">\n<a class=""text-b...",Li,NaN,NaN
1,1,"<li class=""mr-3 float-left"">\n<a class=""text-b...",Li,NaN,NaN
2,2,"<li class=""mr-3 float-left"">\n<a class=""text-b...",Li,NaN,NaN
3,3,"<li class=""breadcrumb-item"">\n<a class=""btn bt...",Li,NaN,NaN
4,4,"<li class=""breadcrumb-item"">\n<a class=""btn bt...",Li,NaN,NaN
...,...,...,...,...,...
5226,5226,"<li class=""grid__item"">\n<link href=""//thecabi...",Product,2 Drawer TV Stand M1,£239.59 GBP
5227,5227,"<li class=""grid__item"">\n<link href=""//thecabi...",Product,Mid Quad Bookcase M1,£0.00 GBP
5228,5228,"<li class=""grid__item"">\n<link href=""//thecabi...",Product,Low Quad Bookcase M1,£181.76 GBP
5229,5229,"<li class=""grid__item"">\n<link href=""//thecabi...",Product,Low 6 Drawer Chest M2,£364.62 GBP


In [71]:
X = df['html']
y = df['category']

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=22)

In [75]:
#tdidf turns textual data into numerical format
pipeMNB = Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())])
pipeCNB = Pipeline([('tfidf',TfidfVectorizer()),('clf',ComplementNB())])
pipeSVC = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [77]:
#training mnb
pipeMNB.fit(X_train, y_train)
predictMNB = pipeMNB.predict(X_test)
print(f"MNB score is: "+str(accuracy_score(y_test,predictMNB)))

MNB score is: 0.9121298949379179


In [78]:
#training cnb
pipeCNB.fit(X_train, y_train)
predictCNB = pipeCNB.predict(X_test)
print(f"CNB score is: "+str(accuracy_score(y_test,predictCNB)))

CNB score is: 0.9140401146131805


In [79]:
#training svc
pipeSVC.fit(X_train, y_train)
predictSVC = pipeSVC.predict(X_test)
print(f"SVC score is: "+str(accuracy_score(y_test,predictSVC)))

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVC score is: 0.9598853868194842


In [80]:
#check accuracy of models
print(classification_report(y_test,predictSVC))

              precision    recall  f1-score   support

         Div       0.96      0.92      0.94       354
          Li       0.91      0.96      0.93       295
     Product       1.00      1.00      1.00       398

    accuracy                           0.96      1047
   macro avg       0.96      0.96      0.96      1047
weighted avg       0.96      0.96      0.96      1047



In [81]:
test = """<li class="grid__item">
<link href="//thecabinetshop.co.uk/cdn/shop/t/18/assets/component-rating.css?v=24573085263941240431657786494" media="all" rel="stylesheet" type="text/css">


</div>
</link></li>
"""

In [83]:
pipeSVC.predict([test])


array(['Li'], dtype=object)

In [84]:
pipeMNB.predict([test])

array(['Product'], dtype='<U7')

In [85]:
pipeCNB.predict([test])

array(['Product'], dtype='<U7')

In [86]:
url = "https://www.tekshop.co.uk/laptops/home-laptops"
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')
products = []
productGrid = soup.find("ol",class_="products list items product-items") 
prs = productGrid.contents
test = prs[0].next_sibling
len(prs)

25

In [111]:
counter=0
identifiedProducts = []
for child in prs:
    if pipeSVC.predict([str(child)]) == ['Product']:
        price = child.find(string=re.compile("£"))
        name = child.find(class_='name').text
        productJSON = {"html":child, "name":name.strip(),"price":price.strip()}
        identifiedProducts.append(productJSON)
len(identifiedProducts)

12

In [125]:
saveAs = 'Tekshop'
with open('/Users/Tico/Documents/Masters/LaMona/ConfirmedProducts/'+saveAs+'.csv', 'w', newline='') as csvfile:
    fieldnames = ['html', 'name', 'price']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for item in identifiedProducts:
        writer.writerow(item)

In [113]:
#IGNORE BELOW THIS LINE

In [33]:
products=[]
for child in productGrid:
    if pipeMNB.predict([str(child)]) == ['Product']:
        products.append(child)

In [35]:
products=[]
for child in productGrid:
    products.append(pipeCNB.predict([str(child)]))
len(products)

25

In [37]:
divs = soup.find_all('div')
identifiedProducts = []
for div in divs:
    divSubContent = div.contents
    if divSubContent != None:
        if pipeSVC.predict([str(divSubContent)]) == ['Product']:
            print(type(divSubContent))
            break
    

<class 'list'>
